In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import xlsxwriter

**Motivos para falsificar facturaciones**
* Sobre facturación: para fugar capitales o para blanquear capitales
* Sub facturación: para evadir impuestos

In [3]:
df_producto=pd.read_csv('../data/EXPO_DOC_12019000.csv', encoding='latin-1',sep=';', dtype={'CUIT':'str',
                                                                                            'dia':'str',
                                                                                            'mes':'str',
                                                                                            'anio':'str'})
df_producto.pais_descri=df_producto.pais_descri.apply(lambda x: x.capitalize())
df_producto.empresa=df_producto.empresa.apply(lambda x: x.lower())
df_producto['fob_unitario_ton']=df_producto.fob/df_producto.pnet*1000
bm=pd.read_excel('../data/CMO-Historical-Data-Monthly.xlsx',sheet_name='Monthly Prices',skiprows=6)
bm=bm[[bm.columns[0],bm.columns[24]]].rename(columns={'Unnamed: 0':'Fecha', 'SOYBEANS':'soja_bm'})
bm['anio']=bm.Fecha.apply(lambda x: x[:4])
bm['mes']=bm.Fecha.apply(lambda x: x[5:])
bm.drop('Fecha',axis=1,inplace=True)

df_producto=pd.merge(left=df_producto, right=bm, how='left', on=['anio','mes'])
df_producto['diferencia_ref']=df_producto.fob_unitario_ton-df_producto.soja_bm

#Capitalizo los precios al valor del ultimo mes para que se puedan comparar.
df_producto['fob_unitario_ton_capit']=df_producto.fob_unitario_ton*df_producto.soja_bm.iloc[-1]/df_producto.soja_bm
df_producto['diferencia_ref_capi']=df_producto.fob_unitario_ton_capit-df_producto.soja_bm.iloc[-1]
df_producto['fecha']=df_producto["mes"].astype(str)+'-'+df_producto['anio'].astype(str)
df_producto['fecha']=pd.to_datetime(df_producto["fecha"]).dt.strftime('%m-%Y')
df_producto=df_producto.sort_values(['anio','mes','dia'], ascending=True)

desde=df_producto['anio'][0]
hasta=df_producto['anio'].iloc[-1]

In [4]:
df_producto_outliers=df_producto.sort_values('fob_unitario_ton',ascending=False)
df_producto_outliers=df_producto_outliers[['anio','dia','mes','CUIT','empresa','pais_descri','fob','pnet','fob_unitario_ton','soja_bm']]
df_producto_outliers

,anio,dia,mes,CUIT,empresa,pais_descri,fob,pnet,fob_unitario_ton,soja_bm
464,2022,14,02,30716309122,trigus trading s.a.,Uruguay,61954.40,144.08,430000.0,661.63
327,2022,10,02,30716309122,trigus trading s.a.,Uruguay,61799.60,143.72,430000.0,661.63
440,2022,13,04,30716891182,jm y cia s.a.s.,Chile,76500.00,17000.00,4500.0,720.79
194,2022,06,07,30630720911,petroagro sociedad anonima,Estados unidos,142642.50,150150.00,950.0,678.20
479,2022,14,07,30630720911,petroagro sociedad anonima,Estados unidos,7298983.00,7683140.00,950.0,678.20
...,...,...,...,...,...,...,...,...,...,...
802,2022,24,06,30715372130,c y g exportaciones s.r.l.,Paraguay,96000.00,300000.00,320.0,737.06
42,2022,02,06,30715372130,c y g exportaciones s.r.l.,Paraguay,96000.00,300000.00,320.0,737.06
98,2021,03,11,23217189969,cagnoni walter ernesto,Uruguay,119269.92,387240.00,308.0,551.04
250,2021,08,01,20334234038,correa yonathan luis,Brasil,18900.00,84000.00,225.0,576.37


In [5]:
#Hay problemas con Uruguay, 3 envios (2 de trigus trading s.a), el peso neto es incorrecto
df_producto=df_producto[df_producto.diferencia_ref_capi<3000].reset_index()

In [6]:
df_producto

,index,dia,mes,anio,CUIT,empresa,CNRO_ENMIENDA,NOMEN,nomen_descri,pais,pais_descri,fob,pnet,fob_unitario_ton,soja_bm,diferencia_ref,fob_unitario_ton_capit,diferencia_ref_capi,fecha
0,101,04,01,2021,30711778493,auqui s.a.,6,12019000,Porotos de soja excluidos p/siembra,208,Chile,69798.94,141580.0,493.0,576.37,-83.37,501.065982,-84.734018,01-2021
1,127,05,01,2021,30710824580,a.c.b. alimentos coronel baigorria s.a.,6,12019000,Porotos de soja excluidos p/siembra,208,Chile,164172.50,338500.0,485.0,576.37,-91.37,492.935094,-92.864906,01-2021
2,167,06,01,2021,30683950129,carolina vasquetto y elio marinelli sociedad s...,6,12019000,Porotos de soja excluidos p/siembra,208,Chile,57400.00,140000.0,410.0,576.37,-166.37,416.708017,-169.091983,01-2021
3,205,07,01,2021,30711778493,auqui s.a.,6,12019000,Porotos de soja excluidos p/siembra,208,Chile,69720.06,141420.0,493.0,576.37,-83.37,501.065982,-84.734018,01-2021
4,250,08,01,2021,20334234038,correa yonathan luis,6,12019000,Porotos de soja excluidos p/siembra,203,Brasil,18900.00,84000.0,225.0,576.37,-351.37,228.681229,-357.118771,01-2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1028,30,07,2022,30716296950,izco,6,12019000,Porotos de soja excluidos p/siembra,212,Estados unidos,93548.00,133640.0,700.0,678.20,21.80,604.629903,18.829903,07-2022
1061,1029,30,07,2022,30716590514,campos verdes argentinos s.a.,6,12019000,Porotos de soja excluidos p/siembra,212,Estados unidos,46440.00,64500.0,720.0,678.20,41.80,621.905043,36.105043,07-2022
1062,1061,31,07,2022,30708321334,sociedad agricola e inmobiliaria las piedras l...,6,12019000,Porotos de soja excluidos p/siembra,208,Chile,141624.00,252000.0,562.0,678.20,-116.20,485.431436,-100.368564,07-2022
1063,1062,31,07,2022,30709436755,soyapampa s.a.,6,12019000,Porotos de soja excluidos p/siembra,212,Estados unidos,31822.00,45460.0,700.0,678.20,21.80,604.629903,18.829903,07-2022


In [7]:
# desde=df_producto.mes[0]+'/'+'01/'+df_producto.anio[0]
# hasta=str(int(df_producto.mes.iloc[-1])+1)+'/'+'01/'+df_producto.anio.iloc[-1]
# monthDates = pd.DataFrame({
#     'Fecha': pd.date_range(start=desde, end=hasta, freq='M').strftime('%m-%Y')
# })

In [8]:
def plot_precio_soja(y=['fob_unitario_ton', 'diferencia_ref'], color=['pais_descri','empresa']):
     if y=='fob_unitario_ton':
          title_1= 'Fob unitario'
          title_2= 'FOB por tonelada exportada de soja <br> <sup> Precio de referencia del Banco Mundial'
     elif y=='diferencia_ref':
          title_1= 'Diferencia en USD'
          title_2= 'Diferencia con precio de referencia en USD por tonelada exportada de soja <br> <sup> Precio de referencia del Banco Mundial'
     precio_soja_plot=px.scatter(
     df_producto.round(1),
     x='fecha',
     y=y,
     color=color,
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'soja_bm': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'soja_bm',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_plot.update_yaxes(title_text= title_1, 
                                   range=[df_producto[y].min()-100,df_producto[y].max()+100]
                                   )
     precio_soja_plot.update_xaxes(title_text='')

     precio_soja_plot.update_traces(marker=dict(size=12,
                                   line=dict(width=2,
                                             color='DarkSlateGrey')),
                    selector=dict(mode='markers'))

     precio_soja_plot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=700, width=900,
                                   template = 'none',
                                   title_text=title_2)

     return precio_soja_plot

def precio_soja_boxplot_mes():
     precio_soja_boxplot=px.box(
     df_producto.round(1),
     x='fecha',
     y='fob_unitario_ton',
     color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'soja_bm': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'soja_bm',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= 'Fob unitario', 
                                   range=[df_producto['fob_unitario_ton'].min()-100,df_producto['fob_unitario_ton'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=700, width=1000,
                                   template = 'none',
                                   title_text='FOB por tonelada exportada de soja <br> <sup> Precio de referencia del Banco Mundial',
                                   showlegend=False)

     return precio_soja_boxplot

def precio_soja_boxplot_capitalizado():
     precio_soja_boxplot=px.box(
     df_producto.round(1),
     # x='fecha',
     x='fob_unitario_ton_capit',
     # color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob_unitario_ton_capit':'Precio capitalizado',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'soja_bm': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'fob_unitario_ton_capit',
                    'soja_bm',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= '', 
                                   # range=[df_producto['fob_unitario_ton_capit'].min()-100,df_producto['fob_unitario_ton_capit'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='Fob unitario')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=400, width=700,
                                   template = 'none',
                                   title_text=f'FOB por tonelada exportada de soja {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia del Banco Mundial',
                                   showlegend=False)

     return precio_soja_boxplot

def precio_soja_boxplot():
     precio_soja_boxplot=px.box(
     df_producto.round(1),
     # x='fecha',
     x='fob_unitario_ton',
     # color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'soja_bm': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'soja_bm',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= '', 
                                   # range=[df_producto['fob_unitario_ton'].min()-100,df_producto['fob_unitario_ton'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='Fob unitario')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=400, width=700,
                                   template = 'none',
                                   title_text=f'FOB por tonelada exportada de soja {desde}-{hasta} <br> <sup> Precio de referencia del Banco Mundial',
                                   showlegend=False)

     return precio_soja_boxplot

precio_soja_boxplot_capitalizado()
# Acá hay algo para ver, no detecta a las expo paraguayas como outliers... Sesgo a mostrar a los valores superiores.
# plot_precio_soja('fob_unitario_ton')

In [9]:
plot_precio_soja(y='diferencia_ref',color='pais_descri').write_html('../output/diferencia con ref soja pais.html')
plot_precio_soja(y='fob_unitario_ton',color='pais_descri').write_html('../output/fob unitario por ton soja pais.html')
plot_precio_soja(y='diferencia_ref',color='empresa').write_html('../output/diferencia con ref soja empresa.html')
plot_precio_soja(y='fob_unitario_ton',color='empresa').write_html('../output/fob unitario por ton soja empresa.html')
precio_soja_boxplot().write_html('../output//Expo boxplot sin capitalizar.html')
precio_soja_boxplot_capitalizado().write_html('../output//Expo boxplot capitalizado.html')

## Empresas exportadoras Frecuencia Mensual

In [10]:
gb = df_producto.groupby(['anio','mes','NOMEN','nomen_descri'])
counts = gb.size().to_frame(name='n_operaciones')
df_mensual=(counts
 .join(gb.agg({'CUIT': 'nunique'}).rename(columns={'CUIT': 'n_empresas'}))
 .join(gb.agg({'fob': 'sum'}).rename(columns={'fob': 'fob'}))
 .join(gb.agg({'pnet': 'sum'}).rename(columns={'pnet': 'pnet'}))
 .reset_index()
)

df_mensual['fob_unitario_ton']=df_mensual.fob/df_mensual.pnet*1000

df_mensual_refer=pd.merge(left=df_mensual,right=bm,how='left', on=['anio','mes'])
df_mensual_refer['diferencia_ref']=df_mensual_refer.fob_unitario_ton-df_mensual_refer.soja_bm
df_mensual_refer['fob_unitario_ton_capit']=df_mensual_refer.fob_unitario_ton*df_mensual_refer.soja_bm.iloc[-1]/df_mensual_refer.soja_bm
df_mensual_refer['diferencia_ref_capi']=df_mensual_refer.fob_unitario_ton_capit-df_mensual_refer.soja_bm.iloc[-1]

df_mensual_refer

,anio,mes,NOMEN,nomen_descri,n_operaciones,n_empresas,fob,pnet,fob_unitario_ton,soja_bm,diferencia_ref,fob_unitario_ton_capit,diferencia_ref_capi
0,2021,01,12019000,Porotos de soja excluidos p/siembra,41,11,4.907442e+06,1.110724e+07,441.823737,576.37,-134.546263,519.882816,-158.317184
1,2021,02,12019000,Porotos de soja excluidos p/siembra,41,12,4.992713e+06,1.105199e+07,451.747910,578.26,-126.512090,529.822973,-148.377027
2,2021,03,12019000,Porotos de soja excluidos p/siembra,66,13,7.303955e+06,1.568027e+07,465.805419,585.71,-119.904581,539.361177,-138.838823
3,2021,04,12019000,Porotos de soja excluidos p/siembra,56,19,2.211573e+08,4.413564e+08,501.085602,597.13,-96.044398,569.116030,-109.083970
4,2021,05,12019000,Porotos de soja excluidos p/siembra,100,32,4.571470e+08,8.833659e+08,517.505855,646.80,-129.294145,542.629052,-135.570948
5,2021,06,12019000,Porotos de soja excluidos p/siembra,87,39,2.737664e+08,5.161844e+08,530.365484,614.68,-84.314516,585.172564,-93.027436
6,2021,07,12019000,Porotos de soja excluidos p/siembra,89,34,4.085644e+08,7.966469e+08,512.855080,600.44,-87.584920,579.272393,-98.927607
7,2021,08,12019000,Porotos de soja excluidos p/siembra,100,39,7.611149e+08,1.465977e+09,519.186182,585.80,-66.613818,601.078983,-77.121017
8,2021,09,12019000,Porotos de soja excluidos p/siembra,99,33,4.433330e+08,8.331424e+08,532.121522,557.55,-25.428478,647.268973,-30.931027
9,2021,10,12019000,Porotos de soja excluidos p/siembra,60,22,1.891789e+08,3.556341e+08,531.948059,551.95,-20.001941,653.622926,-24.577074


### Tabla 1
Los valores son a precios capitalizados

In [11]:
contar_paises=df_producto.groupby('pais_descri').size().sort_values(ascending=False)
paises_interesantes=pd.DataFrame(contar_paises).reset_index().pais_descri.unique()[:7]
paises_interesantes

array(['Chile', 'Estados unidos', 'China', 'Uruguay', 'Brasil', 'Canada',
       'Paraguay'], dtype=object)

In [16]:
def datos_tabla_paises(pais):
    return [len(df_producto[df_producto.pais_descri==pais]),round(df_producto[df_producto.pais_descri==pais].fob_unitario_ton_capit.min(),1), round(df_producto[df_producto.pais_descri==pais].fob_unitario_ton_capit.quantile(0.25),1),round(df_producto[df_producto.pais_descri==pais].fob_unitario_ton_capit.quantile(0.5),1),round(df_producto[df_producto.pais_descri==pais].fob_unitario_ton_capit.quantile(0.75),1),round(df_producto[df_producto.pais_descri==pais].fob_unitario_ton_capit.min(),1)]

# A precios capitalizados
dataFrameData = {
    pais:datos_tabla_paises(pais) for pais in paises_interesantes
}

tabla_1_expo=pd.DataFrame({
    '':['No. of trade records','Lowest price','Lower quartile price','Median price','Upper quartile price','Highest price'],
    'Mundo':[len(df_producto),round(df_producto.fob_unitario_ton_capit.min(),1), round(df_producto.fob_unitario_ton_capit.quantile(0.25),1),round(df_producto.fob_unitario_ton_capit.quantile(0.5),1),round(df_producto.fob_unitario_ton_capit.quantile(0.75),1),round(df_producto.fob_unitario_ton_capit.min(),1)],
    **dataFrameData
})

# tabla_1_expo=tabla_1_expo.transpose().reset_index()
# header=tabla_1_expo.iloc[0]
# tabla_1_expo=tabla_1_expo[1:]
# tabla_1_expo.columns=header
# tabla_1_expo['No. of trade records']=tabla_1_expo['No. of trade records'].astype(int)
# tabla_1_expo.rename(columns={'':'Paises'},inplace=True)

In [17]:
tabla_1_expo

,,Mundo,Chile,Estados unidos,China,Uruguay,Brasil,Canada,Paraguay
0,No. of trade records,1065.0,399.0,169.0,150.0,132.0,61.0,36.0,29.0
1,Lowest price,203.3,364.1,401.6,370.2,327.4,203.3,435.6,254.3
2,Lower quartile price,430.1,422.5,556.3,495.0,396.4,379.9,509.2,354.2
3,Median price,488.2,457.8,598.0,522.2,437.8,410.1,572.8,365.8
4,Upper quartile price,543.3,506.7,637.5,551.2,486.5,415.3,646.2,373.9
5,Highest price,203.3,364.1,401.6,370.2,327.4,203.3,435.6,254.3


In [12]:
precio_soja_boxplot_capitalizado()

In [13]:
posibles_falsificaciones=df_producto[(df_producto.fob_unitario_ton_capit<df_producto.fob_unitario_ton_capit.quantile(.25)) |
            (df_producto.fob_unitario_ton_capit>df_producto.fob_unitario_ton_capit.quantile(.75))]

posibles_falsificaciones=posibles_falsificaciones.drop('index',axis=1).reset_index(drop=True)
# posibles_falsificaciones.drop('index',axis=1,inplace=)
posibles_falsificaciones

,dia,mes,anio,CUIT,empresa,CNRO_ENMIENDA,NOMEN,nomen_descri,pais,pais_descri,fob,pnet,fob_unitario_ton,soja_bm,diferencia_ref,fob_unitario_ton_capit,diferencia_ref_capi,fecha
0,06,01,2021,30683950129,carolina vasquetto y elio marinelli sociedad s...,6,12019000,Porotos de soja excluidos p/siembra,208,Chile,57400.0,140000.0,410.0,576.37,-166.37,416.708017,-169.091983,01-2021
1,08,01,2021,20334234038,correa yonathan luis,6,12019000,Porotos de soja excluidos p/siembra,203,Brasil,18900.0,84000.0,225.0,576.37,-351.37,228.681229,-357.118771,01-2021
2,08,01,2021,30715281429,agata s.r.l.,6,12019000,Porotos de soja excluidos p/siembra,203,Brasil,28000.0,140000.0,200.0,576.37,-376.37,203.272204,-382.527796,01-2021
3,09,01,2021,30683950129,carolina vasquetto y elio marinelli sociedad s...,6,12019000,Porotos de soja excluidos p/siembra,208,Chile,33472.0,83680.0,400.0,576.37,-176.37,406.544407,-179.255593,01-2021
4,13,01,2021,30683950129,carolina vasquetto y elio marinelli sociedad s...,6,12019000,Porotos de soja excluidos p/siembra,208,Chile,57400.0,140000.0,410.0,576.37,-166.37,416.708017,-169.091983,01-2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,30,07,2022,30711440069,miska s.a.,6,12019000,Porotos de soja excluidos p/siembra,212,Estados unidos,284158.0,405940.0,700.0,678.20,21.80,604.629903,18.829903,07-2022
526,30,07,2022,30716296950,izco,6,12019000,Porotos de soja excluidos p/siembra,212,Estados unidos,93548.0,133640.0,700.0,678.20,21.80,604.629903,18.829903,07-2022
527,30,07,2022,30716590514,campos verdes argentinos s.a.,6,12019000,Porotos de soja excluidos p/siembra,212,Estados unidos,46440.0,64500.0,720.0,678.20,41.80,621.905043,36.105043,07-2022
528,31,07,2022,30709436755,soyapampa s.a.,6,12019000,Porotos de soja excluidos p/siembra,212,Estados unidos,31822.0,45460.0,700.0,678.20,21.80,604.629903,18.829903,07-2022


In [14]:
writer = pd.ExcelWriter(f'../output/exportaciones soja.xlsx', engine='xlsxwriter')
tabla_1_expo.to_excel(writer, sheet_name='tabla_1_expo', index=False)
df_mensual_refer.to_excel(writer, sheet_name='agrupado mensual', index=False)
df_producto.to_excel(writer, sheet_name='desagregado', index=False)
df_producto_outliers.to_excel(writer, sheet_name='outliers', index=False)
posibles_falsificaciones.to_excel(writer, sheet_name='posibles_falsificaciones')
writer.save()